### Input Data
$N$: List of nodes inside a small cluster

$D_{ij}$: Distance between node $i$ and node $j$

$g_j$: Distance between node $j$ and local hub 

$d_i$: Customer demand of node $i$

### Decision variables

$x_{ij} = \mathbb{1}\{\text{if node } i \text{ is on the same path with node } j \}$

$y_{j} = \mathbb{1}\{\text{if node } j \text{ is the first node on a path}  \}$

$C1_j$: Number of Type C Vans needed for the transportation between local hub and first node $j$

$C2_{i}$: Number of Type C Vans needed for the transportation between first node and second node $i$

### Constraints
 - Node i will only be assigned to node j if j is a first node.
 - Each node is only assigned to one first node.
$$
x_{ij} <= y_{j} \ \text{ for all }  i, j \in N \\
\sum_{j \in N} x_{ij} = 1 \ \text{ for each node } i \in N \\
$$
 - Each path has at most 2 nodes.
$$
\sum_{i \in N} x_{ij} <= 2 \ \text{ for each node } j \in N
$$
 - Number of vehicles can fulfill the demand.
$$ 40 C1_j >= \sum_{i \in N} x_{ij} \cdot d_i \text{ for each first node } j \in N$$
$$ 40 C2_{i} >= d_i \text{ for each node } i \in N$$

### Cost Decomposition
1. Cost of Type C1 Vans 
 - Unit cost: 6 * dist
$$ \sum y_j \cdot 6 g_{j} \cdot C1_j$$

2. Cost of Type C2 Vans
 - Unit cost: 6 * dist
$$ \sum x_{ij} \cdot 6 D_{ij} \cdot C2_i$$

### Objective Function
$$ Minimize \sum_{i,j \in N}x_{ij} \cdot 6 D_{ij} \cdot C2_i + \sum_{j \in N}y_j \cdot 6g_j \cdot C1_j$$

In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import math

## Data Preparation

In [2]:
D = pd.read_csv("Data\d_matrix.csv", index_col = 0)

D

,SF0001,SF0002,SF0003,SF0004,SF0005,SF0006,SF0007,SF0008,SF0009,SF0010,...,SF2014,SF2015,SF2016,SF2017,SF2018,SF2019,SF2020,SF2021,SFA,SFB
node_1,,,,,,,,,,,,,,,,,,,,,
SF0001,0.000,0.432,0.478,0.681,0.576,0.214,0.182,0.106,0.082,0.259,...,44.514,23.531,23.568,23.957,21.169,21.078,28.957,28.895,38.550,13.290
SF0002,0.432,0.000,0.149,0.300,0.145,0.242,0.550,0.369,0.453,0.690,...,44.853,23.734,23.765,24.158,21.600,21.509,29.119,29.068,38.844,13.657
SF0003,0.478,0.149,0.000,0.404,0.174,0.264,0.550,0.389,0.473,0.723,...,44.780,23.615,23.645,24.038,21.614,21.521,28.992,28.943,38.752,13.753
SF0004,0.681,0.300,0.404,0.000,0.241,0.528,0.828,0.644,0.722,0.935,...,45.152,24.019,24.049,24.442,21.829,21.742,29.394,29.346,39.144,13.737
SF0005,0.576,0.145,0.174,0.241,0.000,0.378,0.685,0.507,0.593,0.833,...,44.953,23.783,23.813,24.206,21.742,21.651,29.154,29.107,38.927,13.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF2019,21.078,21.509,21.521,21.742,21.651,21.274,20.971,21.144,21.058,20.820,...,29.489,21.719,22.018,22.110,0.434,0.000,27.277,26.675,27.164,13.119
SF2020,28.957,29.119,28.992,29.394,29.154,28.964,28.778,28.910,28.876,28.842,...,24.160,6.045,5.856,5.586,27.708,27.277,0.000,0.803,15.509,34.005
SF2021,28.895,29.068,28.943,29.346,29.107,28.909,28.715,28.851,28.815,28.774,...,23.459,5.704,5.553,5.242,27.107,26.675,0.803,0.000,14.835,33.638


In [3]:
demand = pd.read_excel("Data\demand.xlsx", index_col = 0)

d = demand.demand
d

customer_code
SF0001      6
SF0002    111
SF0003     37
SF0004     20
SF0005      9
         ... 
SF2017     69
SF2018     63
SF2019     51
SF2020     63
SF2021     82
Name: demand, Length: 2021, dtype: int64

### Model

In [5]:
def gsolver(node, D, g, d):
    model1 = pe.ConcreteModel()
    model1.x = pe.Var(node, node, domain = pe.Binary)
    model1.y = pe.Var(node, domain = pe.Binary)
    model1.c1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.c2 = pe.Var(node, domain=pe.NonNegativeIntegers)

    model1.costs = pe.Objective(
        expr = 
        sum(model1.x[i,j] * 6 * D.loc[i,j] * model1.c2[i] for i in node for j in node) + 
        sum(model1.y[j] * 6 * g[j] * model1.c1[j] for j in node),
        sense = pe.minimize)

    # A node can only be assigned to a hub node
    def rule_1(mod, i, j):
        return mod.x[i,j] <= mod.y[j]
    model1.const1 = pe.Constraint(node, node, rule = rule_1)

    # Each node is assigned to 1 and only 1 hub
    def rule_2(mod, i):
        return sum(mod.x[i, j] for j in node) == 1
    model1.const2 = pe.Constraint(node, rule = rule_2)

    # Each path has at most 2 nodes.
    def rule_5(mod, j):
        return sum(mod.x[i, j] for i in node) <= 2
    model1.const5 = pe.Constraint(node, rule = rule_5)
    
    # Number of Type C2 Vans
    def rule_3(mod, i):
        return 40 * mod.c2[i] >= d[i]
    model1.const3 = pe.Constraint(node, rule = rule_3)

    # Number of Type C1 Vans
    def rule_4(mod, j):
        return 40 * mod.c1[j] >= sum(mod.x[i,j] * d[i] for i in node)
    model1.const4 = pe.Constraint(node, rule = rule_4)

    solver = pe.SolverFactory('gurobi')
    result = solver.solve(model1)
    
    return model1

In [8]:
# calculating type C cost in base model
def old_cost(node, i):
    cost = 0
    for a in node:
        original_c = D.loc[i][a] * 6 * math.ceil(d[a] / 40)
        cost += original_c
    return cost

In [34]:
def get_result(cname, result):
    firsts = pd.DataFrame(columns=["local_hub","first_node","C1van"])
    total_c = 0
    total_old_c = 0
    
    nodes = result.sum()
    hub = list(nodes[nodes > 0].index)
    for i in hub:
        node = list(result[result[i]>0][i].index)
        node.remove(i)

        g = D[i]
        model1 = gsolver(node, D, g, d)
        c = model1.costs()
        old_c = old_cost(node, i)
        total_c += c
        total_old_c += old_c

        first = []
        c1van = []
        for j in node:
            if round(model1.y[j]()) == 1:
                if model1.c1[j]() > 0:
                    first.append(j)
                    c1van.append(model1.c1[j]())
        first = pd.DataFrame({"local_hub":i, "first_node":first, "C1van":c1van})
        firsts = pd.concat([firsts, first])

        for j in node:
            for k in node:
                df1.loc[j][k] = round(model1.x[j,k]())


    total_c = round(total_c,2)
    total_old_c = round(total_old_c, 2)
    print(cname, total_c, total_old_c)
    
    return (total_c, total_old_c, firsts)

In [79]:
%%time

df_cost = pd.DataFrame(columns = ['new_cost', 'old_cost'])
df1 = pd.DataFrame(columns = list(d.index), index = list(d.index))
for n in range(1, 21):
    cname = "cluster" + str(n)
    path = "Data\Data_clusters_nodes\\" + cname + "result_node.csv"
    result = pd.read_csv(path, index_col = 0)
    c1, c2, firsts= get_result(cname, result)
    firsts.to_csv(cname + "result_bc_first.csv")
    df_cost.loc[cname] = [c1, c2]
df1.to_csv("result_bc_node.csv")

cluster1 772.2 889.63
cluster2 1127.32 1198.51
cluster3 1392.72 1513.64
cluster4 1070.3 1194.95
cluster5 952.47 1031.99
cluster6 850.12 979.79
cluster7 915.89 1027.65
cluster8 828.37 923.57
cluster9 1253.77 1365.69
cluster10 658.52 719.83
cluster11 1577.57 1734.27
cluster12 966.05 1024.87
cluster13 643.25 756.37
cluster14 989.43 1069.85
cluster15 825.64 899.49
cluster16 1368.54 1520.93
cluster17 869.4 957.13
cluster18 727.31 805.57
cluster19 848.27 958.18
cluster20 585.16 679.51


In [80]:
df_cost

,new_cost,old_cost
cluster1,772.20,889.63
cluster2,1127.32,1198.51
cluster3,1392.72,1513.64
cluster4,1070.30,1194.95
cluster5,952.47,1031.99
cluster6,850.12,979.79
cluster7,915.89,1027.65
cluster8,828.37,923.57
cluster9,1253.77,1365.69
cluster10,658.52,719.83


In [81]:
df_cost.sum()

new_cost    19222.30
old_cost    21251.42
dtype: float64